In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Build Vertex AI Experiment lineage for custom training

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>


## Overview

As a Data Scientist, you want to be able to reuse code path (data preprocessing, feature engineering etc...) that others within your team have written to simplify and standardize all the complex data wrangling. 

Learn more about [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments) and [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata).

### Objective

In this notebook, you learn how to integrate preprocessing code in a Vertex AI experiments. Also you build the experiment lineage lets you record, analyze, debug, and audit metadata and artifacts produced along your ML journey.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex ML Metadata
- Vertex AI Experiments

The steps performed include:

- Execute module for preprocessing data
  - Create a dataset artifact
  - Log parameters
-  Execute module for training the model
  - Log parameters
  - Create model artifact
  - Assign tracking lineage to dataset, model and parameters

### Dataset

This dataset is the UCI News Aggregator Data Set which contains 422,937 news collected between March 10th, 2014 and August 10th, 2014. Below are example records from the dataset:


|ID |TITLE                                                                |URL                                                                                                                        |PUBLISHER        |CATEGORY|STORY                        |HOSTNAME           |TIMESTAMP    |
|---|---------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-----------------|--------|-----------------------------|-------------------|-------------|
|1  |Fed official says weak data caused by weather, should not slow taper |http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss|Los Angeles Times|b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.latimes.com    |1394470370698|
|2  |Fed's Charles Plosser sees high bar for change in pace of tapering   |http://www.livemint.com/Politics/H2EvwJSK2VE6OF7iK1g3PP/Feds-Charles-Plosser-sees-high-bar-for-change-in-pace-of-ta.html   |Livemint         |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.livemint.com   |1394470371207|
|3  |US open: Stocks fall after Fed official hints at accelerated tapering|http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436                |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371550|
|4  |Fed risks falling 'behind the curve', Charles Plosser says           |http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-plosser-says-294430                             |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371793|
|5  |Fed's Plosser: Nasty Weather Has Curbed Job Growth                   |http://www.moneynews.com/Economy/federal-reserve-charles-plosser-weather-job-growth/2014/03/10/id/557011                   |Moneynews        |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.moneynews.com  |1394470372027|

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [ ]:
! pip3 install --upgrade --quiet joblib fsspec gcsfs scikit-learn 
! pip3 install --upgrade --quiet google-cloud-aiplatform 

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### Set data folder

In [ ]:
DATA_PATH = "data"
! mkdir -m 777 -p {DATA_PATH}

### Get the data

In [ ]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
! wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
! mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
! unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
! mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip

! gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

### Import libraries

In [ ]:
# General
import logging

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import collections
import tempfile
import time
import uuid
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Define constants

In [ ]:
import os

# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{uuid.uuid1()}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
TARGET_LABELS = ["b", "t", "e", "m"]
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{uuid.uuid1()}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
SERVE_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn", framework_version="1.0", accelerator="cpu"
)

### Initialize Experiment run

In [ ]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

## Model experimentation with pre-built data pre-processing code

### Data preprocessing

In this step, you simulate some preprocessing on raw data to create the training dataset.

Indeed it would happen that you have some data preprocessing that others within your team may have written. For that reason, you need a way to integrate the preprocessing code in your experiment runs to standardize and reuse all the complex data wrangling that you are dealing with. 

Using Vertex AI Experiments, you can track that code as part of the run execution by adding one `with` statement in your experiment context. 

#### Create a Dataset Metadata Artifact

First you create the Dataset artifact to track the dataset resource in the Vertex ML Metadata and create the experiment lineage.

In [ ]:
raw_dataset_artifact = vertex_ai.Artifact.create(
    schema_title="system.Dataset", display_name=DATASET_NAME, uri=DATASET_URI
)

#### Create a preprocess module

Next, you build a simple preprocessing module to convert the text case and remove the punctuation.

In [ ]:
"""
Preprocess module
"""


import string

import pandas as pd


def preprocess(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """
    Preprocess text
    Args:
        df: The DataFrame to preprocesss
        text_col: The text column name
    Returns:
        preprocessed_df: The datafrane with text in lowercase and without punctuation
    """
    preprocessed_df = df.copy()
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(lambda x: x.lower())
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(
        lambda x: x.translate(str.maketrans("", "", string.punctuation))
    )
    return preprocessed_df

#### Add the `preprocessing` Execution

Vertex AI Experiments supports tracking both executions and artifacts. Executions are steps in an ML workflow that can include but are not limited to data preprocessing, training, and model evaluation. Executions can consume artifacts such as datasets and produce artifacts such as models.

You add the preprocessing step to track its execution in the lineage associated to Vertex AI Experiment. 
For Vertex AI, the parameters are passed inside the message field which we see in the logs. These structures of the logs are predefined.

In [ ]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=PREPROCESS_EXECUTION_NAME
) as exc:
    logging.info(f"Start {PREPROCESS_EXECUTION_NAME} execution.")
    exc.assign_input_artifacts([raw_dataset_artifact])

    # Log preprocessing params --------------------------------------------------
    logging.info("Log preprocessing params.")
    vertex_ai.log_params(
        {
            "delimiter": DELIMITER,
            "features": dumps(COLUMN_NAMES),
            "index_col": INDEX_COL,
        }
    )

    # Preprocessing ------------------------------------------------------------
    logging.info("Preprocessing.")
    raw_df = pd.read_csv(
        raw_dataset_artifact.uri,
        delimiter=DELIMITER,
        names=COLUMN_NAMES,
        index_col=INDEX_COL,
    )
    preprocessed_df = preprocess(raw_df, "title")
    preprocessed_df.to_csv(PREPROCESSED_DATASET_URI, sep=",")

    # Log preprocessing metrics and store dataset artifact ---------------------
    logging.info(f"Log preprocessing metrics and {PREPROCESSED_DATASET_NAME} dataset.")
    vertex_ai.log_metrics(
        {
            "n_records": preprocessed_df.shape[0],
            "n_columns": preprocessed_df.shape[1],
        },
    )

    preprocessed_dataset_metadata = vertex_ai.Artifact.create(
        schema_title="system.Dataset",
        display_name=PREPROCESSED_DATASET_NAME,
        uri=PREPROCESSED_DATASET_URI,
    )
    exc.assign_output_artifacts([preprocessed_dataset_metadata])

### Model Training

In this step, you train a multinomial Naive Bayes pipeline.

#### Create model training module

Below the training module.

**get_training_split :** It takes parameters like  x(The data to be split), y(The labels to be split), test_size(The proportion of the data to be reserved for testing) and random_state(The seed used by the random number generator).
This function return training data, testing data , The training labels and The testing labels.

**get_pipeline :** It return's the model.

**train_pipeline :** It train the model by using model, training data, training lables and return's the trained model.

**evaluate_model :** It evaluate the model and return the accuracy of the model.


In [ ]:
"""
Train module
"""

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


def get_training_split(
    x: pd.DataFrame, y: pd.Series, test_size: float, random_state: int
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    """
    Splits data into training and testing sets
    Args:
        x: The data to be split
        y: The labels to be split
        test_size: The proportion of the data to be reserved for testing
        random_state: The seed used by the random number generator
    Returns:
        x_train: The training data
        x_test: The testing data
        y_train: The training labels
        y_test: The testing labels
    """

    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    return x_train, x_val, y_train, y_val


def get_pipeline():
    """
    Get the model
    Args:
        None
    Returns:
        model: The model
    """
    model = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", MultinomialNB()),
        ]
    )
    return model


def train_pipeline(model: Pipeline, X_train: pd.Series, y_train: pd.Series) -> Pipeline:
    """
    Train the model
    Args:
        model: The model to train
        X_train: The training data
        y_train: The training labels
    Returns:
        model: The trained model
    """
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: Pipeline, X_test: pd.Series, y_test: pd.Series) -> float:
    """
    Evaluate the model
    Args:
        model: The model to evaluate
        X_test: The testing data
        y_test: The testing labels
    Returns:
        score: The accuracy of the model
    """
    # Evaluate model
    y_pred = model.predict(X_test)

    # Store evaluation metrics
    summary_metrics = {
        "accuracy": round(accuracy_score(y_test, y_pred), 5),
        "precision": round(precision_score(y_test, y_pred, average="weighted"), 5),
        "recall": round(recall_score(y_test, y_pred, average="weighted"), 5),
    }
    classification_metrics = {
        "matrix": confusion_matrix(y_test, y_pred, labels=TARGET_LABELS).tolist(),
        "labels": TARGET_LABELS,
    }

    return summary_metrics, classification_metrics


def save_model(model: Pipeline, save_path: str) -> int:
    try:
        with tempfile.NamedTemporaryFile() as tmp:
            joblib.dump(trained_pipeline, filename=tmp.name)
            ! gsutil cp {tmp.name} {save_path}/model.joblib
    except RuntimeError as error:
        print(error)
    return 1

#### Add the `train` Execution

Next, you add the training task to the experiment execution to update the experiment lineage.

In [ ]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=TRAIN_EXECUTION_NAME
) as exc:

    exc.assign_input_artifacts([preprocessed_dataset_metadata])

    # Get training and testing data
    logging.info("Get training and testing data.")
    x_train, x_val, y_train, y_val = get_training_split(
        preprocessed_df[FEATURES],
        preprocessed_df[TARGET],
        test_size=TEST_SIZE,
        random_state=SEED,
    )
    # Get model pipeline
    logging.info("Get model pipeline.")
    pipeline = get_pipeline()

    # Log training param -------------------------------------------------------

    # Log data parameters
    logging.info("Log data parameters.")
    vertex_ai.log_params(
        {
            "target": TARGET,
            "features": FEATURES,
            "test_size": TEST_SIZE,
            "random_state": SEED,
        }
    )

    # Log pipeline parameters
    logging.info("Log pipeline parameters.")
    vertex_ai.log_params(
        {
            "pipeline_steps": dumps(
                {step[0]: str(step[1].__class__.__name__) for step in pipeline.steps}
            )
        }
    )

    # Training -----------------------------------------------------------------

    # Train model pipeline
    logging.info("Train model pipeline.")
    train_start = time.time()
    trained_pipeline = train_pipeline(pipeline, x_train, y_train)
    train_end = time.time()

    # Evaluate model
    logging.info("Evaluate model.")
    summary_metrics, classification_metrics = evaluate_model(
        trained_pipeline, x_val, y_val
    )

    # Log training metrics and store model artifact ----------------------------

    # Log training metrics
    logging.info("Log training metrics.")
    vertex_ai.log_metrics(summary_metrics)
    vertex_ai.log_classification_metrics(
        labels=classification_metrics["labels"],
        matrix=classification_metrics["matrix"],
        display_name="my-confusion-matrix",
    )

    # Generate first ten predictions
    logging.info("Generate prediction sample.")
    prediction_sample = trained_pipeline.predict(x_val)[:10]
    print("prediction sample:", prediction_sample)

    # Upload Model on Vertex AI
    logging.info("Upload Model on Vertex AI.")
    loaded = save_model(trained_pipeline, TRAINED_MODEL_URI)
    if loaded:
        model = vertex_ai.Model.upload(
            serving_container_image_uri=SERVE_IMAGE,
            artifact_uri=TRAINED_MODEL_URI,
            display_name=MODEL_NAME,
        )

    exc.assign_output_artifacts([model])

### Stop Experiment run

In [ ]:
run.end_run()

### Visualize Experiment Lineage

Below you get the link to Vertex AI Metadata UI in the console that show the experiment lineage.

In [ ]:
print("Open the following link:", exc.get_output_artifacts()[0].lineage_console_uri)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

# Delete model
model_list = vertex_ai.Model.list(filter=f'display_name="{MODEL_NAME}"')
for model in model_list:
    model.delete()

# Delete dataset
for dataset_name in [DATASET_NAME, PREPROCESSED_DATASET_NAME]:
    dataset_list = vertex_ai.TabularDataset.list(
        filter=f'display_name="{dataset_name}"'
    )
    for dataset in dataset_list:
        dataset.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

!rm -Rf {DATA_PATH}